In [ ]:
# Prérequis : Installer les modules présents dans le notebook recap

In [ ]:
# Utiliser les embeddings d'HuggingFace
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
# Comparer les embeddings d'une question et des metadata

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("bdconsignes.txt")
pages_txt=loader.load()

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [ ]:
md_header_splits = markdown_splitter.split_text(pages_txt[0].page_content)
# Les metadata sont dans les md_header_splits[i].metadata pour i parcourant l'ensemble des chunks
metadata_docs=[]
for i in range(len(md_header_splits)):
    metadata_inter=""
    if any(item in list(md_header_splits[i].metadata.keys()) for item in ['Header 2','Header 3']) is False:
        metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 1']
    if 'Header 2' in list(md_header_splits[i].metadata.keys()):
        metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 2']
    if 'Header 3' in list(md_header_splits[i].metadata.keys()):
        metadata_inter=metadata_inter+'. '+md_header_splits[i].metadata['Header 3']
    metadata_docs.append(metadata_inter)    

In [ ]:
embeddings_metadata = model.encode(metadata_docs)

In [ ]:
print(embeddings_metadata[0][0:5])

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
query="Comment changer d'APE ?"

In [ ]:
# Calculate the similarity between the query embedding and document embeddings
query_embedding = model.encode(query)
similarity_scores = cosine_similarity(query_embedding.reshape(1,-1), embeddings_metadata)

In [ ]:
print(similarity_scores)

In [ ]:
sorted_indices = np.argsort(similarity_scores)[::-1]
print(sorted_indices)
print(type(sorted_indices))
print(type(sorted_indices[0]))

In [ ]:
md_header_splits[2]

In [ ]:
sorted_documents = [md_header_splits[i] for i in sorted_indices[0]]

In [ ]:
for i in range(len(md_header_splits)):
    print(metadata_docs[i])
    print(similarity_scores[0][i],'\n')

In [ ]:
print(type(md_header_splits[1].metadata))

In [ ]:
print(md_header_splits[1].metadata['Header 2'])

In [ ]:
for i in range(len(md_header_splits)):
    if 'Header 3' in list(md_header_splits[i].metadata.keys()):
        print(md_header_splits[i].metadata['Header 3'])

In [ ]:
# Tout est rassemblé dans une fonction
def met_match(question, md_header_splits,k):
# Les metadata sont dans les md_header_splits[i].metadata pour i parcourant l'ensemble des chunks
    metadata_docs=[]
    for i in range(len(md_header_splits)):
        metadata_inter=""
        if any(item in list(md_header_splits[i].metadata.keys()) for item in ['Header 2','Header 3']) is False:
            metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 1']
        if 'Header 2' in list(md_header_splits[i].metadata.keys()):
            metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 2']
        if 'Header 3' in list(md_header_splits[i].metadata.keys()):
            metadata_inter=metadata_inter+'. '+md_header_splits[i].metadata['Header 3']
        metadata_docs.append(metadata_inter)
    embeddings_metadata = model.encode(metadata_docs)
    query_embedding = model.encode(question)
    similarity_scores = cosine_similarity(query_embedding.reshape(1,-1), embeddings_metadata)
    # Sort the documents based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[0]
    sorted_documents = [md_header_splits[i].page_content for i in sorted_indices][::-1]
    top_k_documents=""
    for i in range(k):
        top_k_documents = top_k_documents+'\n'+sorted_documents[i]
    return top_k_documents

In [ ]:
met_match(md_header_splits=md_header_splits,k=3,question="comment changer d'APE ?")

In [ ]:
print(similarity_scores)

In [ ]:
# Simple chain

In [ ]:
# On crée un prompt template, on initialise le modèle et l'output parser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import streamlit as st
# Build prompt llama chat
template_chat = """<s>[INST] <<SYS>>
\n
Vous êtes un assistant conversationnel cordial et honnête, qui répond, uniquement en langue française, aux questions ou aux problèmes posés par un usager. Si vous ne connaissez pas la réponse, répondez simplement que vous ne savez pas, n'essayez pas d'inventer la réponse. 
\n<</SYS>>
\n
À l'aide du contexte ci-dessous, répondez, uniquement en langue française, au problème suivant posé par un usager : {question}
\n\n
Contexte : 
\n
{context}
[/INST]"""
QA_CHAIN_PROMPT_chat = PromptTemplate.from_template(template_chat)

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    # Verbose is required to pass to the callback manager
n_batch = 512
llm = LlamaCpp(
    model_path='./llama-2-7b-chat.Q5_K_M.gguf',
    n_gpu_layers=0,
    max_tokens = 8000,
    temperature = 0.1,
    n_batch=n_batch,
    f16_kv=True,
    use_mlock=True,
    n_ctx=2048,
    callback_manager=callback_manager,
    n_threads=8,
    verbose=True,
    streaming=True)

In [ ]:
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
chain = RunnableMap({
    "context": lambda x: met_match(loader=loader,k=2,question=x["question"]),
    "question": lambda x: x["question"]
}) | QA_CHAIN_PROMPT_chat | llm | output_parser

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
# On utilise config-callbacks pour avoir le détail du déroulement de la chaîne
chain.invoke({"question": "Comment changer d'APE?"},config={'callbacks': [ConsoleCallbackHandler()]})

In [ ]:
# Ajout de la mémoire

In [ ]:
template_memory=PromptTemplate(input_variables=['chat_history', 'question'],
               template='''<s>[INST] <<SYS>>
\n
Vous êtes un assistant conversationnel cordial et honnête, qui répond, 
               uniquement en langue française, aux questions ou aux problèmes posés par un usager. 
               Si vous ne connaissez pas la réponse, répondez simplement que vous ne savez pas, 
               n'essayez pas d'inventer la réponse.
               \n<</SYS>>\n
               Historique de la conversation:\n{chat_history}
               \n À l'aide de l'historique de la conversation ci-dessus, et du contexte ci-dessous, répondez, 
               uniquement en langue française, au problème suivant posé par un usager :
               \nSuite de la conversation: {question}\n[/INST]''')

In [ ]:
# On garde en mémoire l'historique des messages
# Return_messages = True signifie qu'on met les messages passés sous forme de liste, 
# et non de la forme d'un simple texte

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True)

In [ ]:
# Le module ConversationalRetrievalChain gère la mémoire

retriever=vectordb.as_retriever()
qa_memory = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    condense_question_prompt=template_memory
)